In [1]:
import sklearn as sk
from sklearn.linear_model import LinearRegression,Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV,cross_val_score
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
from xgboost import XGBRegressor
from keras import backend as K
from keras.models import Sequential
from keras.datasets import mnist
from keras.layers import Dense, Activation, Dropout
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
import keras.backend as K
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.regularizers import l2
from sklearn.linear_model import ElasticNet
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('HW3Data.csv')
df.shape

df.head(5)

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,127.87
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0.00
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127.48
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0.00
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0.00


In [3]:
list(df)

['sequence_number',
 'US',
 'source_a',
 'source_c',
 'source_b',
 'source_d',
 'source_e',
 'source_m',
 'source_o',
 'source_h',
 'source_r',
 'source_s',
 'source_t',
 'source_u',
 'source_p',
 'source_x',
 'source_w',
 'Freq',
 'last_update_days_ago',
 '1st_update_days_ago',
 'Web order',
 'Gender=male',
 'Address_is_res',
 'Purchase',
 'Spending']

In [4]:
df.isnull().values.any()

False

The variables 1st_update_days_ago and last_update_days_ago are highly collinear hence we look to form a new feature update_duration by subtracting the values of last_update_days_ago from 1st_update_days_ago 

In [5]:
df['update_duration'] = df['1st_update_days_ago'] - df['last_update_days_ago']

In [6]:
df_data = df[['Freq','update_duration','Web order','Gender=male', 'Address_is_res','US', 'source_a'
              , 'source_b','source_c', 'source_d','source_e', 'source_m', 'source_o', 'source_h',
             'source_r', 'source_s','source_t', 'source_u', 'source_p', 'source_x', 'source_w']]


df_data.head()

,Freq,update_duration,Web order,Gender=male,Address_is_res,US,source_a,source_b,source_c,source_d,...,source_m,source_o,source_h,source_r,source_s,source_t,source_u,source_p,source_x,source_w
0,2,0,1,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,31,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df_target = df[['Spending']]
df_target.head(5)

,Spending
0,127.87
1,0.00
2,127.48
3,0.00
4,0.00


## TEST TRAIN SPLIT

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_data,df_target, test_size = 0.1,random_state = 55)

## Normalisation


In [9]:
mm_scaler = MinMaxScaler()
X_train = mm_scaler.fit_transform(X_train)
X_test = mm_scaler.fit_transform(X_test)

pd.DataFrame(X_test[:5,])

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.066667,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.066667,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We will set up our code to enter the models we want to run and also the hyperparameters we want to tune the models on.
We run 8 models on the data and caluclate the MAE and RMSE to select the best model. 
We have used Grid Serach CV to tune all the models

In [10]:
random_state_models = 64
svm = SVR()
knn = KNeighborsRegressor()
dtc = DecisionTreeRegressor(random_state = random_state_models)
lreg = LinearRegression()
ridgereg = Ridge(normalize=True)
lassoreg = Lasso(normalize=True, max_iter=1e5)
enet = ElasticNet()
xgb = XGBRegressor()
ab = AdaBoostRegressor()
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()

p_grid = {1: {"model": lreg, "hyperparam":{'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}},
          2: {"model": ridgereg, "hyperparam": {'alpha':[1,0.1,0.01,0.001,0.0001]}},
          3: {"model": lassoreg ,"hyperparam":{'alpha':[1,0.1,0.01,0.001,0.0001]}},
          4: {"model": knn ,"hyperparam":{"n_neighbors": list(range(1,15)), "weights": ["uniform", "distance"]}},5:{"model":dtc,"hyperparam":{"criterion": ["entropy","gini"], "max_depth": list(range(1,15)),"min_samples_leaf": range(5, 15), "min_samples_split": range(5, 15)}},
          5: {"model": dtc,"hyperparam":{ "max_depth": list(range(1,15)),"min_samples_leaf": range(5, 15), "min_samples_split": range(5, 15)}},
          6: {"model": svm ,"hyperparam":{'kernel': ['rbf','linear'], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}},
          7: {"model": enet,"hyperparam":{'alpha':np.logspace(-5,2,8),'l1_ratio':[.2,.4,.6,.8]}},
          8: {"model": xgb,"hyperparam": {'eta': [0.0001, 0.001, 0.01,1], 'gamma':[0,1, 10],'colsample_bytree':[0.5,0.7,1],'eval_metric': ['mae'], 'subsample':[.5,.8,1], 'verbosity':[0]}},
          9: {"model": ab,"hyperparam": {'n_estimators': range(5,100,5),'learning_rate' : [0.01,0.05,0.1,0.3,0.5,1],'loss' : ['linear', 'square', 'exponential']}},
          10:{"model": rf,"hyperparam": {'bootstrap': [True],'max_depth': list(range(2,10)),'n_estimators': [10,20,50,100,200]}},
          11:{"model": gb,"hyperparam":{'n_estimators': range(10,100,10),'learning_rate' : [0.01,0.05,0.1,0.3,0.5,1],'max_depth': range(2,8,2),'min_samples_split': range(2,8,2)}}}
 

Next we tune all the models using  Grid Cross Validation.

In [11]:
performance_metrix = {}


# Non_nested parameter search and scoring

for m, param_grid in enumerate(p_grid.values()):
    clf = RandomizedSearchCV(estimator=param_grid["model"], param_distributions=param_grid["hyperparam"],scoring = 'neg_mean_absolute_error', cv=10)
    clf.fit(X_train,y_train)

    y_pred = clf.predict(X_test)


    performance_metrix[m] = {'model': (str(param_grid["model"])[:5]), 'param':clf.best_params_,'scores':{'R2':r2_score(y_test,y_pred),
                                      'MAE':mean_absolute_error(y_test,y_pred),'RMSE':sqrt(mean_squared_error(y_test,y_pred))}}
    performance_metrix[m]

Running Neural Networks

In [12]:
def NN_build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21,kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21, kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21, kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21, kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21, kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=200, activation = 'relu', input_dim=21, kernel_regularizer = l2(0.01)))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_absolute_error',  metrics=['mae','accuracy','mse'])
    return regressor


In [13]:
regressor = KerasRegressor(build_fn=NN_build_regressor, batch_size=59,epochs=100,verbose =0)    

model = regressor.fit(X_train, y_train, epochs=50)

y_pred= regressor.predict(X_test)

performance_metrix[11] = {'model': 'NN','scores':{'R2':r2_score(y_test,y_pred),
                                      'MAE':mean_absolute_error(y_test,y_pred),'RMSE':sqrt(mean_squared_error(y_test,y_pred))}}

Making the score matrix to understand the performance of models.

In [14]:
matrix1 = {"Linear Regression": performance_metrix[0]["scores"],
          "Lasso Regression": performance_metrix[1]["scores"],
           "Ridge Regression": performance_metrix[2]["scores"],
          "Elastic Net": performance_metrix[6]["scores"],
          "Knn": performance_metrix[3]["scores"],
         "SVM": performance_metrix[5]["scores"],
          "XGBoost ": performance_metrix[7]["scores"],
          "Ada Boost": performance_metrix[8]["scores"],
          "Random Forest": performance_metrix[9]["scores"],
          "Gradient Boost": performance_metrix[10]["scores"],
          "Neural Network": performance_metrix[11]["scores"]
          }
measuring_martix = pd.DataFrame(data = matrix1).transpose()
measuring_martix

,MAE,R2,RMSE
Linear Regression,76.305538,0.447315,159.774808
Lasso Regression,74.689177,0.443527,160.321490
Ridge Regression,74.420979,0.441558,160.604842
Elastic Net,75.759307,0.446506,159.891796
Knn,81.615696,0.243111,186.975930
SVM,73.444579,0.381593,169.007855
XGBoost,70.634335,0.493391,152.969914
Ada Boost,78.602270,0.466738,156.942216
Random Forest,72.562934,0.456682,158.415104
Gradient Boost,73.722651,0.472106,156.150300


Here we are using MAE to calcualte the best model. We see that with a MAE of 68.83 in Neural Networks gives us the least error hence we choose Neural Networks to model this data.